In [10]:
%pip install tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [13]:
# Set up data paths
train_dir = 'data/train/'
test_dir = 'data/test/'

In [14]:
# Image data generator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize image pixel values to [0, 1]
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift image width
    height_shift_range=0.2,  # Randomly shift image height
    shear_range=0.2,  # Apply shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Randomly flip the image
    fill_mode='nearest'  # Filling pixels when transformations are done
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [15]:
# Create generators for training and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to a common size
    batch_size=32,
    class_mode='categorical'  # Since it's a multiclass classification
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 1211 images belonging to 4 classes.
Found 99 images belonging to 4 classes.


In [16]:
# Generate the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for multiclass
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
%pip install scipy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 15s 376ms/step - accuracy: 0.3025 - loss: 1.4562 - val_accuracy: 0.4167 - val_loss: 1.3556
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4375 - loss: 1.2034 - val_accuracy: 0.5104 - val_loss: 1.2839
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 356ms/step - accuracy: 0.4611 - loss: 1.1866 - val_accuracy: 0.5729 - val_loss: 1.2293
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6875 - loss: 1.0364 - val_accuracy: 0.5104 - val_loss: 1.3621
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 334ms/step - accuracy: 0.5875 - loss: 1.0636 - val_accuracy: 0.6250 - val_loss: 1.1769
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6562 - loss: 0.9679 - val_accuracy: 0.6042 - val_loss: 1.2504
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 340ms/step - accuracy: 0.5812 - loss: 1.0219 - val_accuracy: 0.5312 - val_loss: 1.1209
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5938 - loss: 0.9214 - val_accuracy: 0.

In [19]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.6259 - loss: 1.0762
Test accuracy: 0.5959596037864685


In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_snake(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions[0])
    class_label = list(train_generator.class_indices.keys())[class_idx]
    return class_label

# Example usage
result = predict_snake(r'data\test\Ahaetulla nasuta\605c4af31346465ead0a1d59f110ed8c.jpg')
print(f'The snake species is: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
The snake species is: Ahaetulla nasuta
